In [7]:
list(range(0,42,4))

[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40]

In [8]:
list(range(4,44,4))

[4, 8, 12, 16, 20, 24, 28, 32, 36, 40]

In [9]:
def shifted_arr_search(shiftArr, num):
    idx = -1  
    
    while len(shiftArr) >= 0:  
        
        if len(shiftArr)>0:

            idx += 1
            print(idx)
            
            if shiftArr[0] == num: 
                break      
        
            else:
                shiftArr = shiftArr[1:] 
        
        elif len(shiftArr) is 0:
            idx = -1
      
    return idx

In [10]:
shifted_arr_search([2],2)

0


0